***Introduction (REVISED VERSION)***

This document provides a detailed overview of the methods and calculations essential for converting raw sensor currents into accurate gas concentration readings using Alphasense sensors. As leaders in gas detection technology, Alphasense provides robust solutions that are crucial for a variety of applications ranging from environmental monitoring to industrial safety. The focus here is to elucidate the specific equations and algorithms that are fundamental to the sensor data interpretation process, enabling users to transform electrical signals emitted by the sensors into meaningful data.

Contained within, you will find a comprehensive exploration of the mathematical models and conversion formulas as provided by Alphasense, designed to facilitate the accurate calculation of gas concentrations from the raw currents measured by the sensors. This documentation is intended to be a valuable resource for engineers, researchers, and professionals who rely on precise and dependable gas measurement techniques. Through an in-depth understanding of these conversion principles, users can ensure the effective application of Alphasense technology in their respective fields, thereby enhancing the reliability and accuracy of their gas detection systems.

---------------------------------------------------------------------

***Terms definitions***

- WEu = uncorrected raw WE output (mV - coming from BU)
- AEu = uncorrected raw AE output (mV - coming from BU)
- WEc = corrected WE output (Result Output)
- WEe = WE electronic offset on the AFE or ISB (mV - initially provided by Alphasense - source from AvSystem on runtime)
- AEe = AE electronic offset on the AFE or ISB (mV - initially provided by Alphasense - source from AvSystem on runtime)
- WEo = WE sensor zero, i.e. the sensor WE output in zero air (mV - initially provided by Alphasense - source from AvSystem on runtime)
- AEo = AE sensor zero, i.e. the sensor AE output in zero air (mV - initially provided by Alphasense - source from AvSystem on runtime)

- WET = Total WE zero offset
- AET = Total AE zero offset


------------------------------------------------------------------
***Sensorbee Sensors***

- CO-B4
    - Type: B

- H2S-B4
    - Type: B

- NO-B4
    - Type: B

- NO2-B43F
    - Type: B

- OX-B431
    - Type: B

- SO2-B4
    - Type: B


------------------------------------------------------
***Python Source Code Library***

In [ ]:
import unittest
import numpy as np
from typing import List, Optional, Dict

class Converter:
  def __init__(self):

      # points - a temporary static values (currently using Alphasense temperature compensation factors) - these  will be replaced with the datatable that the embedded team is measuring
      # sensitivity - based on the sensor sensitivity found in the Alphasense datasheet
      
      self.gasTypes = {
          'CO-B4': {
             'molecular_weight': 28.010,
             'points': [
                 {'x': -30, 'y': 0.7}, 
                 {'x': -20, 'y': 0.7}, 
                 {'x': -10, 'y': 0.7}, 
                 {'x': 0, 'y': 0.7}, 
                 {'x': 10, 'y': 1}, 
                 {'x': 20, 'y': 3}, 
                 {'x': 30, 'y': 3.5}, 
                 {'x': 40, 'y': 4}, 
                 {'x': 50, 'y': 4.5}
             ],
             'sensitivity': [
                 50,
                 60,
                 70,
                 85,
                 95,
                 100,
                 110,
                 115,
                 118
             ],
             'coefficients': [
                 -1.41025641e-07, 
                 1.05769231e-05, 
                 -1.44522145e-04, 
                 -1.02884615e-02, 
                 1.10479021e+00, 
                 8.13857809e+01
             ]
          },
          'H2S-B4': {
              'molecular_weight': 34.08,
              'points': [
                  {'x': -30, 'y': -0.6}, 
                  {'x': -20, 'y': -0.6}, 
                  {'x': -10, 'y': 0.1}, 
                  {'x': 0, 'y': 0.8}, 
                  {'x': 10, 'y': -0.7}, 
                  {'x': 20, 'y': -2.5}, 
                  {'x': 30, 'y': -2.5}, 
                  {'x': 40, 'y': -2.2}, 
                  {'x': 50, 'y': -1.8}
              ],
              'sensitivity': [
                 78,
                 82,
                 87,
                 92,
                 97,
                 100,
                 103,
                 105,
                 105
             ],
             'coefficients': []
          },
          'NO-B4': {
              'molecular_weight': 30.006,
              'points': [
                  {'x': -30, 'y': 2.9}, 
                  {'x': -20, 'y': 2.9}, 
                  {'x': -10, 'y': 2.2}, 
                  {'x': 0, 'y': 1.8}, 
                  {'x': 10, 'y': 1.7}, 
                  {'x': 20, 'y': 1.6}, 
                  {'x': 30, 'y': 1.5}, 
                  {'x': 40, 'y': 1.4}, 
                  {'x': 50, 'y': 1.3}
              ],
              'sensitivity': [
                 89,
                 91,
                 95,
                 98,
                 99,
                 100,
                 101,
                 100,
                 101
             ],
             'coefficients': [
                 1.21028986e-21, 
                 2.91375291e-06, 
                 -1.77156177e-04, 
                 -7.84382284e-03, 
                 9.33216783e-01, 
                 8.49149184e+01
             ]
          },
          'NO2-B43F': {
              'molecular_weight': 46.0055,
              'points': [
                  {'x': -30, 'y': 1.3}, 
                  {'x': -20, 'y': 1.3}, 
                  {'x': -10, 'y': 1.3}, 
                  {'x': 0, 'y': 1.3}, 
                  {'x': 10, 'y': 1}, 
                  {'x': 20, 'y': 0.6}, 
                  {'x': 30, 'y': 0.4}, 
                  {'x': 40, 'y': 0.2}, 
                  {'x': 50, 'y': -1.5}
              ],
              'sensitivity': [
                 102,
                 89,
                 84,
                 87,
                 92,
                 100,
                 105,
                 108,
                 111
             ],
             'coefficients': [
                 7.69230769e-08, 
                 -2.88461538e-06, 
                 -8.68298368e-05, 
                 -1.99592075e-03, 
                 5.52785548e-01, 
                 9.01759907e+01
             ]
          },
          'OX-B431': {
              'molecular_weight': 48.00,
              'points': [
                  {'x': -30, 'y': 0.9}, 
                  {'x': -20, 'y': 0.9}, 
                  {'x': -10, 'y': 1}, 
                  {'x': 0, 'y': 1.3}, 
                  {'x': 10, 'y': 1.5}, 
                  {'x': 20, 'y': 1.7}, 
                  {'x': 30, 'y': 2}, 
                  {'x': 40, 'y': 2.5}, 
                  {'x': 50, 'y': 3.7}
              ],
              'sensitivity': [
                 72,
                 77,
                 85,
                 92,
                 95,
                 100,
                 101,
                 98,
                 97
             ],
             'coefficients': []
          },
          'SO2-B4': {
              'molecular_weight': 64.066,
              'points': [
                  {'x': -30, 'y': -4}, 
                  {'x': -20, 'y': -4}, 
                  {'x': -10, 'y': -4}, 
                  {'x': 0, 'y': -4}, 
                  {'x': 10, 'y': -4}, 
                  {'x': 20, 'y': 0}, 
                  {'x': 30, 'y': 20}, 
                  {'x': 40, 'y': 140}, 
                  {'x': 50, 'y': 450}
              ],
              'sensitivity': [
                 68,
                 78,
                 88,
                 93,
                 97,
                 100,
                 99,
                 100,
                 98
             ],
             'coefficients': [
                 -7.69230769e-08, 
                 2.53496503e-06, 
                 1.46270396e-04, 
                 -1.12325175e-02, 
                 4.98379953e-01, 
                 9.30687646e+01
             ]
          }
      }
  
  def generate_coeffs(self, factors):
    temperatures = np.array([-30, -20, -10, 0, 10, 20, 30, 40, 50]) #static temperature range base on the AlphaSense Zero background current temperature compensation factors
    degree = len(factors) - 1
    coefficients = np.polyfit(temperatures, factors, degree)
    return coefficients

    
  def evaluate_polynomial(self, factors, x, default_coeff):
    if len(factors) != 9:
        raise ValueError("Invalid number of factors. Exactly 9 factors are required.")

    coefficients = []
    if len(default_coeff) > 0:
        coefficients = default_coeff
    else:
        coefficients = self.generate_coeffs(factors) #generate coefficients first using the temperature compensation factors 

    result = 0
    for index, coeff in enumerate(coefficients):
        power = len(coefficients) - 1 - index
        result += coeff * (x ** power)

    return result;

    
  def linear_interpolation(self, x1: float, y1: float, x2: float, y2: float, x: float) -> float:
    if x1 == x2: # Ensure x1 is not equal to x2 to avoid division by zero
        raise ValueError("x1 and x2 cannot be the same value for interpolation.")
    
    m = (y2 - y1) / (x2 - x1) # Calculate the slope (m)
    y = y1 + m * (x - x1) # Calculate the interpolated y-value
    return y

    
  def interpolate_temperature(self, points: List[Dict[str, float]], temperature: float) -> Optional[float]:
    if len(points) < 2:
        raise ValueError("At least two points are required for interpolation.")
    
    points.sort(key=lambda p: p['x']) # Sort points by their temperature (x) values
    
    p1 = None
    p2 = None
    for i in range(len(points) - 1):
        if points[i]['x'] <= temperature <= points[i + 1]['x']:  # Find the two nearest points where the temperature value lies between
            p1 = points[i]
            p2 = points[i + 1]
            break
    
    if p1 and p2:
        result = self.linear_interpolation(p1['x'], p1['y'], p2['x'], p2['y'], temperature)
        return result
    else:
        raise ValueError("temperature value is out of the range of provided points.")

    
  def convert_gas_ppb_to_ugm3(self, gas_type, gas_ppb_value, temperature_c, pressure_hpa):
    molecular_weight = self.gasTypes[gas_type]['molecular_weight']
    temperature_k = temperature_c + 273.15 # Convert temperature from Celsius to Kelvin
    ugm3 = (gas_ppb_value * molecular_weight / (22.41 * (temperature_k / 273.15) * (1 / (pressure_hpa / 1000))))
    return ugm3
  
    # pressure_kpa = pressure_hpa * 0.1 # convert pressure (hPa) to kPa
    # molecular_weight = self.gasTypes[gas_type]['molecular_weight']
    # R = 8.314 # Ideal gas constant in L·kPa/K·mol
    # temperature_k = temperature_c + 273.15 # Convert temperature from Celsius to Kelvin
    # volume_m3 = (R * temperature_k / pressure_kpa) * 0.001  # Convert from liters to cubic meters (V = nRT/P)
    # density_gm3 = (gas_ppb_value * molecular_weight) / (10**9 * volume_m3)  # Compute density: converting ppb to g/m3
    # density_ugm3 = density_gm3 * 10**6  # Convert g/m3 to µg/m3
    # return density_ugm3

    
  def convert_to_ppb(self, corrected, sensitivity, correction_factor, offset_constant, offset_factor):
    ppb = (corrected / sensitivity) * correction_factor
    offset_ppb = ppb - offset_constant
    factored_ppb = offset_ppb * offset_factor
    return factored_ppb

  def map_points(self, compensations, temp_lookup):
    if len(compensations) != len(temp_lookup):
        raise ValueError("Both arrays must have the same length.")
        
    merged_array = [{'x': x, 'y': y} for x, y in zip(temp_lookup, compensations)] # Merge arrays by index
    return merged_array
    
  def convert_to_target_gas(self, sensor_type, temperature, we_sensor, gain, parameters, no2_ppb = None, no2_sensitivity = None, pressure = 1):
    # validate if sensor_type is valid
    if sensor_type not in self.gasTypes:
        valid_types = ', '.join(self.gasTypes.keys())
        raise ValueError("Unknown gas type: {sensor_type}. Please use one of the following: {valid_types}.")
   
    mv_ppb = (we_sensor * gain) / 1000
    WEc = 0;
    WEt = 0;
    
    # compute WEt
    if 'temp_comp_arr' in parameters and 'temp_lookup_arr' in parameters:
        points = self.map_points(parameters['temp_comp_arr'], parameters['temp_lookup_arr'])
        WEt = self.interpolate_temperature(points, temperature)
    else: 
        points = self.gasTypes[sensor_type]['points']
        WEt = self.interpolate_temperature(points, temperature)
   
    # get correction factor
    sensor_sensitivity = self.gasTypes[sensor_type]['sensitivity']
    coefficients = self.gasTypes[sensor_type]['coefficients']
    temp_sensitivity = self.evaluate_polynomial(sensor_sensitivity, temperature, coefficients)
    correction_factor = 100 / temp_sensitivity
      
    if sensor_type != 'OX-B431':
        WEc = parameters['WEu'] - WEt
    else:
        wec_no2 = (no2_ppb * no2_sensitivity) / 1000
        WEc = parameters['WEu'] - (WEt + wec_no2)

    # offsets
    offset_constant = parameters.get('offset_constant', 0) if parameters.get('offset_constant') else 0
    offset_factor = parameters.get('offset_factor', 1) if parameters.get('offset_factor') else 1
      
    ppb =  self.convert_to_ppb(WEc, mv_ppb, correction_factor, offset_constant, offset_factor)
    ugm3 = self.convert_gas_ppb_to_ugm3(sensor_type, ppb, temperature, pressure)
    
    if (ppb < 0):
        return {'WEc': WEc, 'correctionFactor': correction_factor, 'ppb': 0, 'ugm3': 0, 'raw_ppb': ppb, 'raw_ugm3': ugm3}
    else:
        return {'WEc': WEc, 'correctionFactor': correction_factor, 'ppb': ppb, 'ugm3': ugm3, 'raw_ppb': ppb, 'raw_ugm3': ugm3}
    


--------------------------------------

***Expected Parameters / Sources:***

***Electrodes***:
- ***WEe*** - initially from AlphaSense Calibration file, but will be sourced from AvSystem during runtime
- ***AEe*** - initially from AlphaSense Calibration file, but will be sourced from AvSystem during runtime
- ***WEo*** - initially from AlphaSense Calibration file, but will be sourced from AvSystem during runtime
- ***AEo*** - initially from AlphaSense Calibration file, but will be sourced from AvSystem during runtime
- ***WEu*** - to be sourced from AvSystem as raw voltage during runtime
- ***AEu*** - to be sourced from AvSystem as raw voltage during runtime

***Values / Settings*** 
- ***Temperature*** - to be sourced from AvSystem temperature object
- ***we_sensor (Sensitivity (nA/ppm))*** - initially from AlphaSense Calibration file, but will be sourced from AvSystem during runtime
- ***gain*** - initially from AlphaSense Calibration file, but will be sourced from AvSystem during runtime

If Sensor type is equal to OX-B431 then the following additional parameters are required:
- ***no2_ppb*** - no2 concentration from NO2-B43F. This can be obtained using the same function above, but with the needed parameters for NO2-B43F
- ***no2_sensitivity*** - provided by AlphaSense

---------------------------------------------------------------------------------------





***Test Environment Arguments/Parameters***

Enter Your Arguments/Parameters below (Update the values to see changes in the result):

In [32]:
# source from Gas Sensor Data 2
WEu = 225.9356536865234
AEu = 247.2929840087891
temperature = 5.779575824737549

# source from Gas Sensor Settings 2
WEe = 224 
AEe = 255
AEo = 246
WEo = 222 
we_sensor = -431
gain = -0.73
no2_sensitivity = None
sensor_type = 'NO2-B43F'

# additional parameter for OX/O3
no2_ppb = None

# pressure for ug/m3 computation
pressure = 1000.001220703125
offset_constant = 0
offset_factor = 1


***Test Environment***

In [33]:
class TestConverter(unittest.TestCase):
  def setUp(self):
    self.converter = Converter()

  def test_convert_to_target_gas(self):
    result = self.converter.convert_to_target_gas(sensor_type, temperature, we_sensor, gain, {
       'WEu': WEu, 
       'WEe': WEe, 
       'AEu': AEu, 
       'AEe': AEe,
       'AEo': AEo,
       'WEo': WEo,
       'offset_constant': offset_constant,
       'offset_factor': offset_factor,
       'temp_comp_arr': [226.28,224.29,222.36,219.87,215.94,203.82,171.3],
       'temp_lookup_arr': [-10,0,10,20,30,40,50]
    }, no2_ppb, no2_sensitivity, pressure)
      
    print('result', result)    

unittest.main(argv=[''], verbosity=3, exit=False)

test_convert_to_target_gas (__main__.TestConverter.test_convert_to_target_gas) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


result {'WEc': 2.7611118206977494, 'correctionFactor': 1.071987151166028, 'ppb': 9.407483058578725, 'ugm3': 18.66290062538457, 'raw_ppb': 9.407483058578725, 'raw_ugm3': 18.66290062538457}
